In [ ]:
import numpy as np 
import pandas as pd 

import seaborn as sns
import matplotlib.pyplot as plt

from category_encoders.binary import BinaryEncoder

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error

# Introdução

Neste notebook, atacarei o desafio [Walmart Recruiting - Store Sales Forecasting](https://www.kaggle.com/competitions/walmart-recruiting-store-sales-forecasting/overview) do Kaggle como parte do processo seletivo para o Zé Delivery. Esse desafio consiste em realizar predições de quanto cada departamento de cada loja irá vender em cada semana, com base em vendas passadas e informações sobre as lojas e os locais em que elas estão.

Durante a análise e modelagem, procurarei motivar cada passo dado, trazendo fontes e explicações sobre os conceitos utilizados sempre que possível. Gráficos serão usados para melhor visualização dos dados.

Vale apontar que as técnicas aqui utilizadas serão por vezes simplificadas para que o desafio possa ser terminado em um tempo razoável. Os pontos fracos dessas simplificações serão apontados e as ações que eu tomaria na "situação ideal" serão descritas na seção "O que eu faria para fazer um modelo melhor?" ao fim do notebook.

As predições nos dados de teste serão submetidas ao Kaggle, com seu score privado sendo revelado ao fim do notebook. As aleatoriedades serão eliminadas a partir da escolha de random states fixos de forma que o notebook possa ser 100% reproduzido.

**Obs: usarei os termos "feature", "atributo" e "coluna" sempre me referindo à mesma coisa: as diferentes classes de dados usadas na modelagem do problema. Acho útil colocar isso aqui para evitar confusão ao longo do texto**

# Carregamento e limpeza de dados

Nessa seção carergarei os dados e farei pequenas modificações e adequações para facilitar a codagem

In [ ]:
# Carregando os arquivos providos pelo Kaggle
features_raw = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/features.csv.zip')
train_raw = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/train.csv.zip')
test_raw = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/test.csv.zip')
stores_raw = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/stores.csv')
sample_submission = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip')

Aqui temos os dados "crus", ou seja, diretamente como foram fornecidos. Eles são separados em "treino", que serão os dados que serão analisados para gerar insights para a modelagem e que também servirão para treinar os modelos de machine learning, e "teste", que serão os dados para os quais queremos fazer predições.

É importante que a análise e modelagem seja feito apenas no conjunto de treino, com o teste sendo exclusivamente para realizar a predição. Isso impede que tenhamos um resultado demasiadamente otimista e terminemos com um modelo sem real poder de predizer nada. 

Usar o teste para realizar a modelagem seria como fazer as provas já sabendo as respostas: iria bem na prova, mas seria incapaz de extrapolar para uma situação real que fuja daquelas respostas "decoradas". Essa metodologia de separação entre treino e teste é essencial para o sucesso dos modelos (embora existam outras para outras situações).

Com os dados crus carregados, precisamos juntar as informações contidas nas diferentes tabelas fornecidas

In [ ]:
# Fazendo o merge dos dados
sf = stores_raw.merge(features_raw, on='Store')
train = train_raw.merge(sf, on=['Store', 'Date', 'IsHoliday'])
test = test_raw.merge(sf, on=['Store', 'Date', 'IsHoliday']).sort_values(by = ['Store','Dept','Date']).reset_index(drop=True) # Para submeter o arquivo final, temos que ter os dados de teste na mesma ordem que na tabela sample_submission

In [ ]:
train.head()

Prefiro escrever as colunas todas em letras minúsculas para escrever os códigos com mais agilidade e portanto farei essa mudança.

In [ ]:
# Deixar o nome das colunas em letras minúsculas
train.columns = train.columns.str.lower()
test.columns = test.columns.str.lower()

In [ ]:
train.dtypes

Vemos acima que os dados da coluna 'Date' não estão gravados no tipo de dado mais adequado (datetime). Portanto, faremos essa transformação


In [ ]:
# Transformar as datas no formato datetime do pandas
train.date = pd.to_datetime(train.date)
test.date = pd.to_datetime(test.date)

# Análise exploratória dos dados (EDA)

Como fica explícito no título, nesta seção exploraremos os dados de forma a gerar insights que possam ajudar na modelagem do problema. A partir dela podemos decidir que atributos serão de fatos importantes, ter ideias de como criar novas features, checar se os dados fornecidos fazem sentido, etc.

Como está explicitado na descrição do desafio, trabalharemos com as seguintes colunas:
- Store: identificação de cada uma das diferentes lojas
- Dept: identificação de cada departamento
- Date: data
- Weekly_Sales: vendas semanas. Essa é a nossa coluna alvo, ou seja, a que iremos quere prever
- IsHoliday: marcador que diz se naquela semana houve um feriado importante (SuperBowl, Labor Day, Thanksgiving e Natal)
- Type: não foi identificado no desafio a que essa coluna se refere. Isso será abordado a frente
- Size: tamanho da loja
- Temperature: temperatura média do local onde a loja está
- Fuel_Price: preço médio do combustível na localização da loja
- MarkDown (1 a 5): colunas com dados anonimizados, ou seja, cuja origem não foi explicitada
- CPI: índice de preço do consumidor médio da localização da loja
- Unemployment: índice de desemprego médio da região

O primeiro passo será adicionar uma coluna com a granularidade adequada para as datas. Usaremos semanas, já que os dados são organizados dessa maneira, principalmente nossa variável alvo (weekly sales). Irei fazer uma coluna com o ano também para diferenciar entre semanas de anos diferentes para uma mesma loja.

In [ ]:
# Granularidade de data
train['year'] = train.date.dt.year
train['week'] = train.date.dt.week 

test['year'] = test.date.dt.year
test['week'] = test.date.dt.week 

### Alguns números

Veremos algumas estatísticas básicas dos dados disponíveis, como média, valores máximos/mínimos, etc. Isso servirá como uma checagem de se os dados fazem sentido de  fato e frequentemente geram insights importantes sobre como realizar a modelagem.

In [ ]:
train.describe()

Os valores parecem fazer sentido, já que podemos inferir com certa segurança as unidades usadas para representar as quantidades nas colunas:
- Temperature: em fahrenheits, indo de -2 (-18°C) a 100(38°C)
- Fuel price: litros/galão, com 3.36 de média, o que condiz com [fontes](https://www.eia.gov/dnav/pet/hist/LeafHandler.ashx?n=pet&s=emm_epm0_pte_nus_dpg&f=m)
- CPI: embora esteja um pouco maior que a [média nos EUA para aquele período](https://fred.stlouisfed.org/series/CPIAUCSL), tem valores razoáveis (e temos que lembrar que se referem apenas às áreas onde estão as lojas)
- Unemployment: porcentagens razoáveis considerando a [média nacional de 9% para aquele período](https://tradingeconomics.com/united-states/unemployment-rate#:~:text=Unemployment%20Rate%20in%20the%20United,percent%20in%20May%20of%201953.)

### Vendas ao longo do ano

Vamos plotar os valores médios de vendas semanais para cada ano para ver se verificamos alguma sazonalidade que nos dê algum insight. Isso é especialmente importante devido à grande ênfase dada aos feriados no enunciado deste desafio.

In [ ]:
# Marcadores para super bowl, labor day, thanksgiving e christmas
holiday_weeks = train[train.isholiday==True].week.unique()
holiday_names = ['super bowl', 'labor day', 'thanksgiving', 'christmas']
holiday_colors = ['c', 'y', 'm', 'r']
print(holiday_weeks)

In [ ]:
# Plotar vendas semanais médias para cada ano

years = [2010, 2011, 2012]
plt.figure(figsize=(25, 10))

# Plots por ano
for year in years:
    df_grouped = train[train.year == year].groupby('week')['weekly_sales'].mean()
    sns.lineplot(df_grouped.index, df_grouped.values)

# Marcadores de feriado
for week, name, color in zip(holiday_weeks, holiday_names, holiday_colors):
    plt.axvline(week, color=color, ls='--', label=name)
    plt.legend()

plt.grid(axis='y')
plt.legend(['2010', '2011', '2012'] + holiday_names)
plt.show()

Aqui fica claro que, em termos de vendas, Thanksgiving e Natal são os feriados mais importantes, de longe. Vale observar também que as vendas do Natal são feitas na semana anterior. Sendo assim, na seção de engenharia de atributos (feature engineering) criarei 4 colunas novas que poderão auxiliar na modelagem:

- Marcador da semana de Thanksgiving (binário)
- Marcador da semana pré-Natal (binário)
- Quantidade de dias até o Thanksgiving
- Quantidade de dias até o Natal

# Análise das features

Em contraste com a seção anterior que deu conta de aspectos mais gerais dos dados, aqui abordaremos os diferentes atributos dos dados separadamente, de forma a gerar insights sobre a utilização deles no treino dos modelos de machine learning.

### Stores e departments

Aqui analizaremos o quanto cada loja/departamento são diferentes uns dos outros, de forma a decidir se vale a pena incluir estes atributos na modelagem das vendas. Esta decisão é importante, já que a inclusão de dados categóricos (e não simplesmente numéricos, como "temperatura", por exemplo) como estes trazem uma complexidade maior à modelagem, como veremos a frente.

In [ ]:
# Número de lojas diferentes
train.store.unique().size

In [ ]:
# Distribuição de dados entre as lojas
train.store.value_counts(normalize=True).head()

In [ ]:
# Número de departamentos diferentes
train.dept.unique().size

In [ ]:
# Distribuição de dados entre os departamentos
train.dept.value_counts(normalize=True).head()

Existe um número considerável de lojas (45) e departamentos (81), com os dados sendo bem distribuídos dentre eles. É importante que tenhamos números relevantes de dados de cada tipo, de modo que nenhum fique subrepresentado e gere dstorções na capacidade de predição do modelo.

In [ ]:
# Plotar média de vendas por loja

store_sales = train.groupby('store')['weekly_sales'].mean()
plt.figure(figsize=(25, 15))
sns.barplot(store_sales.index, store_sales.values)
plt.grid(axis='y')
plt.show()

Insight: aparentemente há uma grande disparidade de vendas entre as lojas, o que significa que suas identificações provavelmente serão de grande valor para o modelo de predição de vendas.

In [ ]:
# Plotar média de vendas por departamento

store_dpt = train.groupby('dept')['weekly_sales'].mean()
plt.figure(figsize=(25, 15))
sns.barplot(store_dpt.index, store_dpt.values)
plt.grid(axis='y')
plt.show()

O mesmo vale para os diferentes departamentos.

Além disso, vemos que temos departamentos com valores de venda semanais BEM baixos, o que podem acabar sendo um estorvo para o aprendizado do modelo por serem tão diferentes dos demais. Embora não vá ser feito neste trabalho, verificarei a quantidade de dados deste tipo para explorar a possibilidade de criação de um modelo espcífico para estes dados.

In [ ]:
# Departmentoss com vendas BEM baixas
train[train.dept.isin([28, 39, 43, 45, 47, 51, 54, 60, 77, 78])].size / train.size

Temos uma quantidade razoável de dados deste tipo (5%), o que sinaliza uma possibilidade de tratamento em separado destes dados.

### O que é 'type'?

Tirando as colunas anonimizadas ("markdowns"), a coluna 'type' é a única que não traz qualquer explicação sobre sua natureza. Aqui exploraremos ela para ver se podemos usá-la de alguma forma.

In [ ]:
# Representantes de cada type
train.type.value_counts(normalize=True)

In [ ]:
# Média semanal de vendas de cada tipo
train.groupby('type')['weekly_sales'].mean()

In [ ]:
# Mediana semanal de vendas de cada tipo
train.groupby('type')['weekly_sales'].median()

As lojas com 'type' A não apenas tem mais representantes como também tem maior média/mediana de vendas semanais. O mesmo ocorre para B em relação a C.

Como A > B > C consistentemente, modelaremos esse comportamento atribuindo valores a cada um dos tipos que obedeçam a essa relação: A = 3, B = 2, C = 1

In [ ]:
# Mapeando A, B e C em 1, 2 e 3, respectivamente
train['type'] = train.type.map({'A': 3, 'B': 2, 'C': 1})
test['type'] = test.type.map({'A': 3, 'B': 2, 'C': 1})

Acabamos de tomar nossa primeira decisão de modelagem dos dados a partir de insights gerados por análises! Um ponto importante de perceber é que, embora os insights sejam gerados apenas a partir do conjunto de treino, as mesmas modificações são realizadas também no conjunto de teste. Isso porque o teste deverá ter a mesma forma que o treino de forma que o modelo consiga extrapolar o que aprendeu no algoritmo de machine learning.

### Correlações

Correlações são relações estatísticas entre diferentes atributos que nos ajudam a entender como elas funcionam entre si. Em poucas palavras, atribuiremos um valor entre -1 e 1 a cada par de atributos que nos responderão a seguinte pergunta:

**Se pegamos dados com valores cada vez maiores para o atributo 1, o que devemos esperar para os respectivos valores do atributo 2?**

Valores negativos de correlação indicam que devemos esperar que o atributo 2 tenha valores cada vez menores. O oposto ocorre para números positivos, com suas normas indicando o quão forte é essa relação de crescimento/diminuição. 

Valores próximos a 0 indicam que os atributos tendem a ser independentes, ou seja, que o comportamento de um atributo não deverá trazer nenhuma informação por si só sobre o comportamento de outro atributo. Eles não estarão correlacionados.

Normalmente a correlação é uma forma simples de se ter um feeling sobre que atributos serão mais ou menos importantes para realizarmos nossas predições. Normas altas de correlação com o atributo alvo (no nosso caso, as vendas semanais) podem indicar que temos aí uma coluna valiosa para nossa modelagem. Por outro lado, correlações altas entre atributos que não sejam o alvo podem sugerir que eles serão redundantes, já que possuem comportamento parecido.

Embora eu vá usar os resultados da matriz de correlação para tomar algumas decisões, vale apontar que a interação dos atributos dentro dos modelos de machine learning são em geral muito mais complexos do que as captadas por esta simples relação estatística. No fim, só saberemos se um atributo será útil ou não ao modelo através de testes.

In [ ]:
# Fonte: https://seaborn.pydata.org/examples/many_pairwise_correlations.html

sns.set_theme(style="white")

# Compute the correlation matrix
corr = train.drop(columns=['store', 'dept', 'isholiday']).corr() # exclude categorical features from Pearson corr

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(22, 18))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, 
            annot=True)

Correlações altas: 
- markdowns 1 e 4: markdown4 será descartada, já que tem baixíssima correlação com a feature alvo (weekly_sales). 
- fuel_price e year: fuel_price tem uma correlação extremamente baixa com a feature alvo. Além disso, a coluna 'year' ajuda na diferenciação de semanas de mesmo valor. Portanto, fuel_price será descartada.

Correlações baixas:
- temperature: tem correlação baixíssima com a feature alvo e será descartada.
- unemployment e cpi: também têm correlações relativamente baixas mas, por ora, permanecerão, já que intuitivamente parecem tem uma relação maior com vendas do que temperatura, por exemplo.

Markdowns: 
markdowns 1 e 5 têm as maiores correlações com o atributo alvo e suas permanências serão consideradas. Essa decisão dependerá do quão grande será o número de valores nulos nestas colunas, já que fomos advertidos no anunciado do desafio que essas informações estão incompletas.

**Importante**

Como dito antes, análise de correlação não dão conta da complexidade envolvida nos modelos de machine learning, sendo apenas um primeiro approach. Descartei colunas aqui como uma primeira aposta principalmente para ilustar como um processo de modelagem pode ser guiado por essa ferramenta, mas idealmente voltaria a revisitá-las caso houvesse necessidade de melhoria do modelo.

In [ ]:
# Tirando as colunas mencionadas acima
train = train.drop(columns=['markdown2', 'markdown3', 'markdown4', 'fuel_price', 'temperature'])
test = test.drop(columns=['markdown2', 'markdown3', 'markdown4', 'fuel_price', 'temperature'])

Agora vamos verificar qual a porcentagem de dados nulos temos nas colunas markdown

In [ ]:
na_1_size = train[train.markdown1.isna()].size
na_5_size = train[train.markdown5.isna()].size
train_size = train.size

print('markdown1: \n')
print(na_1_size / train_size, '% de valores nulos\n')

print('markdown5: \n')
print(na_5_size / train_size, '% de valores nulos')

Nós temos mais de 60% de perda de dados nessas colunas! Por agora irei descartá-las, mas veremos mais a frente que temos maneiras de lidar com a existência de valores nulos em nossos dados.

In [ ]:
# Retirando o resto das colunas markdown
train = train.drop(columns=['markdown1', 'markdown5'])
test = test.drop(columns=['markdown1', 'markdown5'])

# Feature engineering

Na seção 'Stores e Departments' decidimos criar novas features novas. Isso será feito agora

In [ ]:
# Marcadores das semanas the thanksgiving e pré-natal
train['is_thanksgiving'] = (train.week == 47).astype(int)
train['is_pre_xmas'] = (train.week == (52 - 1)).astype(int)

test['is_thanksgiving'] = (test.week == 47).astype(int)
test['is_pre_xmas'] = (test.week == (52 - 1)).astype(int)

In [ ]:
# Dias para Thanksgiving
train['days_to_thanksgiving'] = (pd.to_datetime(train.year.astype(str) + '-11-24') - pd.to_datetime(train.date)).dt.days
test['days_to_thanksgiving'] = (pd.to_datetime(test.year.astype(str) + '-11-24') - pd.to_datetime(test.date)).dt.days

# Dias para natal
train['days_to_xmas'] = (pd.to_datetime(train.year.astype(str) + '-12-24') - pd.to_datetime(train.date)).dt.days
test['days_to_xmas'] = (pd.to_datetime(test.year.astype(str) + '-12-24') - pd.to_datetime(test.date)).dt.days

In [ ]:
# Não usarei mais os dados de data cheios, portanto descartarei
train = train.drop(columns='date')
test = test.drop(columns='date')

# Pré-processamento

A parte de pré-processamento é extremamente importante para que possamos apresentar os dados ao algoritmo de machine learning numa linguagem que ele entenda. Lidaremos com dois tipos de pré-processamento aqui:

- **Preenchimento de valores nulos:** Valores nulos não são aceitos pela grande maioria de algoritmos de machine learning. Sendo assim, identificaremos onde existem valores nulos para que possamos substituí-los por valores númericos. 

- **Codificação de variáveis categóricas:** Enquanto colunas como 'week', 'unemployment' e outras têm valores numéricos facilmente interpretados pelo algoritmo de machine learning, as colunas 'store' e 'dept' são nada mais do que códigos de identificação, cujo valor numérico não carrega nenhum valor intrínseco. Afinal, a loja '12' não é maior que a loja '10' ou menor que a loja '15' em nenhum sentido, por exemplo. Desta forma, precisamos transformar essa identificação de lojas e departamentos em uma forma 'legível' para o modelo, o que será feito e explicado em breve





### Preenchimento de valores nulos

In [ ]:
# Quais colunas do treino têm valores nulos?
train.isna().sum()

In [ ]:
# Quais colunas do teste têm valores nulos?
test.isna().sum()

A opção feita aqui é a de substituir os nulos pela média dos valores da respectiva coluna no conjunto de treino, de forma que representem o caso "mais comum" o possível e tenham pouco impacto nas decisões do modelo. Em outros tipos de modelo podemos fazer a substitução por "0", seguindo o mesmo princípio.

In [ ]:
# Preencher nulos com a média
test['cpi'] = test.cpi.fillna(test.cpi.mean())
test['unemployment'] = test.unemployment.fillna(test.unemployment.mean())

### Encoding

Como foi dito, precisamos lidar com as features categóricas de nosso dataset.

A primeira é mais simples: a coluna 'isholiday' que está como booleana (True/False) pode ser facilmente transformada em binária, que é um formato que o algoritmo de machine learning "entende", já que representa simplesmente presença ou ausência. 



In [ ]:
# Transforming 'isholiday' into integers (0 or 1)
train['isholiday'] = train.isholiday.astype(int)
test['isholiday'] = test.isholiday.astype(int)

As colunas 'store' e 'dept' são mais complicadas, com a primeira tendo 45 classes diferentes e a segunda 81. 

Uma forma de codificar isso seria usar o método conhecido como [One-Hot-Encoding](https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/), que cria uma coluna binária para cada uma das classes existentes, bastando atribuir o valor 1 para a coluna referente ao valor nominal da classe. Esse seria um método válido, mas tem a desvantagem de criar muitas colunas adicionais de dados, o que traz seus próprio problemas como a famigerada ["maldição da dimensionalidade"](https://towardsdatascience.com/the-curse-of-dimensionality-50dc6e49aa1e), que aumenta muito a possibilidade de overfitting nos dados de treino.

Uma segunda técnica foi escolhida: mantendo a ideia de transformar dados categóricos em binários, codificamos cada uma das possíveis classes em seu valor binário. Por exemplo, a classe 9 seria codificada como 1001 e assim em diante. Essa metodologia tem a vantagem de só precisar criar o número de colunas referentes à necessidade de representação binária do seu número total de classes: para representar 45 classes precisamos de 6 colunas (45 < 2^6) e para 81 classes precisamos de 7 (81 < 2^7). Esse método é chamado de [Binary Encoding](https://contrib.scikit-learn.org/category_encoders/binary.html).

In [ ]:
# Separando a feature alvo para treinar o encoder apropriadamente
X = train.drop(columns=['weekly_sales'])
y_exp = train.weekly_sales

In [ ]:
# Defino o encoder
bin_enc = BinaryEncoder(cols = ['store', 'dept'])

# Treino e transformação dos dados
X_exp = bin_enc.fit_transform(X)
X_test = bin_enc.transform(test)

In [ ]:
X_exp.head()

In [ ]:
X_exp.columns

# Criando os modelos

Nesta seção criaremos finalmente os modelos. Primeiramente faremos uma rápida escolha do algorítmo usado, para depois realizarmos uma busca de parâmetros simplificada e finalmente o treino do modelo final e a avaliação da importância das features em suas predições.

Toda a parte de experimentação desta seção será feita utilizando um conjunto de validação fixo, este criado a partir da seleção aleatória de 15% dos dados de treino. Esses dados não serão usados para o treino do modelo na parte de experimentação, mas serão incluídos no treinamento do modelo final. Vale apontar que uma forma melhor de realizar essa experimentação seria através do uso de Cross Validation, que diminui as chances de overfitting nos dados de treino, mas evitarei a técnica para termos uma experimentação mais rápida.

### Testando diferentes algoritmos

Como realizar busca e refinamento de parâmetros para múltiplos algoritmos é um processo demorado, utilizarei uma abordagem simplificada. Testarei alguns poucos algoritmos com suas configurações padrões para, a partir dos resultados, escolher apenas 1 deles e seguir com a experimentação.

A métrica utilizada será a mesma definida pelo enunciado do desafio: WMAE (weighted mean absolute error), que usará a coluna 'isholiday' para atribuir peso 5 às semanas com feriados.

In [ ]:
# Simples train/valid split
X_train, X_valid, y_train, y_valid = train_test_split(X_exp, y_exp, random_state=42, test_size=0.15)

In [ ]:
# Vamos usar XGBRegressor, LGBMRegressor e RandomForestRegressor como possíveis algoritmos
regressors = [XGBRegressor(random_state=42), 
              LGBMRegressor(random_state=42), 
              RandomForestRegressor(random_state=42)]

for reg in regressors:
    # Fit model
    model = reg.fit(X_train, y_train)
    
    # Predict
    y_pred_val = model.predict(X_valid)
    
    # Definir métrica do desafio: MAE com peso 5 para feriados
    weights = X_valid.isholiday.apply(lambda x: 5 if x else 1)
    WMAE = mean_absolute_error(y_valid, y_pred_val, sample_weight=weights)
    
    print(f'O WMAE do algoritmo {str(model)} é: {WMAE}')


Random Forest é o melhor algoritmo de longe! Sendo assim, este será o algoritmo usado para realizar refinamento de parâmetros e gerar o modelo final a ser usado nas predições.

### Refinamento de hiperparâmetros

Refinamento de parâmetros será a parte mais computacionalmente pesada deste notebook. Por este motivo, tomarei algumas previdências para simplificar o processo, de forma que ele possa ser feito em um tempo razoável:
- Será feito um [Grid Search](https://towardsdatascience.com/grid-search-for-model-tuning-3319b259367e) em apenas dois parâmetros da RandomForestRegressor: número de estimators (n_estimators) e profundidade máxima das árvores (max_depth). mais parâmetros para refinamento podem ser encontrados na [documentação da RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)
- Como já foi dito, não usarei Cross Validation, já que seu uso multiplicaria o tempo de execução por um fator igual ao número de folds usados.

O Grid Search será codado com 'for's encadeados, para maior clareza. O uso da classe [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) é recomendada, mas por estarmos usando uma métrica com pesos (WMAE), programar seu uso seria desnecessariamente complicado. Uma forma de fazer isso pode ser encontrada nesta [página do Stack Overflow](https://stackoverflow.com/questions/49581104/sklearn-gridsearchcv-not-using-sample-weight-in-score-function). 

In [ ]:
# Valores escolhidos por experimentações anteriores
n_estimators_params = [i for i in range(100, 151, 10)] # Default: 100
max_depth_params = [i for i in range(25, 41, 3)] + [None] # Default None: até todas as folhas estarem "puras"

In [ ]:
# Coding the simple grid-search

weights = X_valid.isholiday.apply(lambda x: 5 if x else 1)
best_est = 100 # Default
best_depth = None # Default
best_WMAE = 1596.69 # Baseline

for est in n_estimators_params:
    for depth in max_depth_params:
        print(f'Training: n_estimators = {est}, max_depth = {depth}')
        # Fit model
        model = RandomForestRegressor(n_estimators=est, max_depth=depth, 
                                      random_state=42).fit(X_train, y_train)

        # Predict
        y_pred_val = model.predict(X_valid)

        WMAE = mean_absolute_error(y_valid, y_pred_val, sample_weight=weights)
        
        if WMAE < best_WMAE:
            best_est = est
            best_depth = depth
            best_WMAE = WMAE
        
        print(f'WMAE = {WMAE}\n')

print(f'Best params: n_estimators = {best_est}, max_depth = {best_depth}\n',
      f'Best WMAE is: {best_WMAE}')

### Importância das features

Agora que os parâmetros estão definidos, treinaremos o modelo final e analisaremos as importâncias de cada uma das features.

In [ ]:
# Treinar com os parâmetros selecionados e com o dataset completo
model = RandomForestRegressor(n_estimators=best_est, 
                              max_depth=best_depth, random_state=42).fit(X_exp, y_exp)

# Gerar dados sobre importancia das features
feature_importances = model.feature_importances_

In [ ]:
# Plotar importancia das features

plt.figure(figsize=(15, 10))
sns.barplot(X_exp.columns, feature_importances)
plt.xticks(rotation=90)
plt.grid(axis='y')
plt.show()

É interessante observar o quão importante as features 'dept' são para o modelo, algo que esperávamos a partir da análise exploratória. O mesmo não se repete com a mesma intensidade com as features 'store', o que indica que a diferenciação entre as lojas pode estar vindo de outra fonte, como a feature 'size' (que tem a maior importância de todas).

As maiores surpresas ficam por conta das features 'cpi' e 'unemployment', que chegaram a ser consideradas para remoção devido às suas baixas correlações com a feature alvo. Isso mostra como análises de correlação nem sempre contam toda a história.

As features criadas para Natal e Thanksgiving se saíram razoavelmente bem, ao contrário da feature 'isholiday', o que pode indicar ter sido uma boa ideia separar os dois feriados mais importantes dos demais.

# Predict

Finalmente, faremos a predição no conjunto de teste e geraremos o .csv que será enviado ao Kaggle para a avaliação final.

In [ ]:
# Predict no teste
y_pred_test = model.predict(X_test)

In [ ]:
# Criar arquivo da predição
sample_submission['Weekly_Sales'] = y_pred_test
sample_submission.to_csv('submission_order.csv',index=False)

# O que eu faria para deixar o modelo melhor?

Ao lidar com modelo de Machine Learning, nos deparamos frequentemente com a decisão entre melhorar o modelo ou levá-lo a produção quando ele já pode agregar algum valor à empresa. Tomar esse tipo de decisão é por si só uma habilidade a ser desenvolvida pelo cientista de dados em vez de estar sempre na busca infinita de um modelo perfeito, mas decidi que seria uma boa ideia listar aqui algumas melhorias que eu tentaria fazer neste modelo se tivesse mais tempo:

- Revisitaria as features descartadas na seção 'Análise de features' já que correlação não diz tudo sobre possíveis contribuições dos atributos. Inclusive as features 'markdown' poderiam ser reconsideradas, mesmo com tamanha porcentagem de nulos, sendo possível realizar a mesma abordagem de preenchimento feita para as colunas 'cpi' e 'unemployment' do conjunto de teste.

- Exploraria a possibilidade de criação de um modelo em separado para departamentos com vendas semanais muito abaixo das demais.

- Usaria Cross Validation para a realização dos experimentos, que melhoraria em muito os problemas de overfitting do modelo treinado.

- Feriados são parte importante da modelagem e podem ser tratados com mais cuidado. Poderíamos por exemplo reunir mais informações sobre os padrões de consumo dos clientes americanos nestes feriados levando em conta os dias da semana, por exemplo. Esse tipo de conhecimento para além dos dados é bastante importante e exemplifica a importância para a  modelagem do conhecimento que pode ser fornecido por PMs ou outros especialistas da empresa.

- Para a busca de parâmetros eu usaria algoritmos mais complexos e mais assertivos (além de procurar mais parâmetros além dos dois usados como exemplo). Uma combinação que eu gosto de usar é a das excelentes classes [HalvingGridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) e [HalvingRandomSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html), que usam um número crescente de recursos (dados) para inferir os melhores parâmetros. Primeiro costumo usar o RandomSearch em um espaço de valores mais extenso, de forma a achar uma região geral onde temos bons resultados. Depois uso o GridSearch em volta deste ponto no espaço de parâmetros para fazer um refinamento, chegando geralmente a excelentes valores. A única preocupação que temos que ter é com mínimos locais, que podem levar a overfitting, mas a componente de CrossValidation dos algoritmos ajudam nisso.

- Para o modelo final, frequentemente é uma boa ideia treinar modelos com algoritmos diversos e fazer um "comitê" final com os melhores, onde os resultados são médias dos modelos individuais. Pode ser uma boa tentativa para este problema.

# Conclusão

Foram utilizadas diversas técnicas de análise e modelagem que, ainda que por vezes simplificadas, mostraram um caminho claro para obtenção de um bom modelo de machine learning para um problema de regressão simples. 

Mesmo com as simplificações, foi obtido um score privado de 4016.10011 com as predições feitas no conjunto de teste, o que deixaria este modelo na 363ª posição no ranking do Kaggle. Tenho confiança que com as melhorias propostas na seção passada poderíamos ter um modelo ainda melhor.